In [1]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

# Load MSA shapefile
msa_shapefile = "../../data/raw/cb_2018_us_cbsa_500k/cb_2018_us_cbsa_500k.shp"
msa_gdf = gpd.read_file(msa_shapefile)

# Load NLCD raster
nlcd_raster = "../../data/raw/nlcd_2021_land_cover_l48_20230630/nlcd_2021_land_cover_l48_20230630.img"
raster = rasterio.open(nlcd_raster)

# Check Coordinate Reference System (CRS)
print("MSA CRS:", msa_gdf.crs)
print("NLCD CRS:", raster.crs)


C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


MSA CRS: EPSG:4269
NLCD CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [2]:
msa_projected = msa_gdf.to_crs(raster.crs)

# Confirm CRS match
print("Reprojected MSA CRS:", msa_projected.crs)

Reprojected MSA CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [3]:
#Clip the NLCD Raster to the MSA Boundaries

from rasterio.mask import mask

# Combine MSA geometries into a single shape
msa_geometry = [msa_projected.geometry.unary_union]

# Mask the NLCD raster with MSA boundaries
with rasterio.open(nlcd_raster) as src:
    out_image, out_transform = mask(src, msa_geometry, crop=True)
    out_meta = src.meta.copy()

# Update metadata for the clipped raster
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})


In [4]:
# Save the clipped raster
clipped_raster_path = "../../data/tidy/clipped_nlcd.tif"
with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
    dest.write(out_image)

print(f"Clipped raster saved at {clipped_raster_path}")


Clipped raster saved at ../../data/tidy/clipped_nlcd.tif


In [5]:
# Compute zonal statistics for each MSA polygon
zonal_statistics = zonal_stats(
    msa_projected,               # Reprojected MSA polygons
    clipped_raster_path,         # Clipped NLCD raster
    stats=["count", "mean", "sum"],  # Compute pixel counts, mean, and sum
    categorical=True,            # Count pixels for each NLCD class
    nodata=0                     # Exclude NoData values
)

# Add zonal statistics to the GeoDataFrame
msa_projected["land_cover_stats"] = zonal_statistics
print(msa_projected[["NAME", "land_cover_stats"]].head())


             NAME                                   land_cover_stats
0   Lynchburg, VA  {11: 71452, 21: 279700, 22: 213434, 23: 58704,...
1  Santa Rosa, CA  {11: 42874, 21: 216600, 22: 137538, 23: 137880...
2   Greenwood, MS  {11: 89715, 21: 96706, 22: 43719, 23: 26221, 2...
3       Tulsa, OK  {11: 582443, 21: 810330, 22: 488479, 23: 32774...
4  Cookeville, TN  {11: 37091, 21: 181261, 22: 96609, 23: 37126, ...


In [6]:
#Step 5: Add Descriptive NLCD Class Names

# NLCD land cover class descriptions
nlcd_mapping = {
    11: "Open Water",
    21: "Developed, Open Space",
    22: "Developed, Low Intensity",
    23: "Developed, Medium Intensity",
    24: "Developed, High Intensity",
    31: "Barren Land (Rock/Sand/Clay)",
    41: "Deciduous Forest",
    42: "Evergreen Forest",
    43: "Mixed Forest",
    51: "Dwarf Scrub",
    52: "Shrubland",
    71: "Grassland/Herbaceous",
    72: "Sedge/Herbaceous",
    73: "Lichens",
    74: "Moss",
    81: "Pasture/Hay",
    82: "Cultivated Crops",
    90: "Woody Wetlands",
    95: "Emergent Herbaceous Wetlands"
}

# Map NLCD codes to descriptions in the statistics
msa_projected["land_cover_descriptions"] = msa_projected["land_cover_stats"].apply(
    lambda stats: {nlcd_mapping[k]: v for k, v in stats.items() if k in nlcd_mapping}
)

# Print the updated GeoDataFrame
print(msa_projected[["NAME", "land_cover_descriptions"]].head())


             NAME                            land_cover_descriptions
0   Lynchburg, VA  {'Open Water': 71452, 'Developed, Open Space':...
1  Santa Rosa, CA  {'Open Water': 42874, 'Developed, Open Space':...
2   Greenwood, MS  {'Open Water': 89715, 'Developed, Open Space':...
3       Tulsa, OK  {'Open Water': 582443, 'Developed, Open Space'...
4  Cookeville, TN  {'Open Water': 37091, 'Developed, Open Space':...


In [7]:
msa_projected

# Assuming your DataFrame is named 'data' #filter out only metropolitan statistical area instead of micropolitan statistical area
msa_projected = msa_projected[msa_projected['LSAD'] == 'M1'].reset_index(drop=True)

msa_projected

,CSAFP,CBSAFP,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,land_cover_stats,land_cover_descriptions
0,None,31340,310M400US31340,31340,"Lynchburg, VA",M1,5491925729,68559452,"POLYGON ((1411159.279 1700825.497, 1411285.802...","{11: 71452, 21: 279700, 22: 213434, 23: 58704,...","{'Open Water': 71452, 'Developed, Open Space':..."
1,488,42220,310M400US42220,42220,"Santa Rosa, CA",M1,4081491107,497455304,"POLYGON ((-2336932.550 2089874.862, -2336646.0...","{11: 42874, 21: 216600, 22: 137538, 23: 137880...","{'Open Water': 42874, 'Developed, Open Space':..."
2,538,46140,310M400US46140,46140,"Tulsa, OK",M1,16238554204,492994165,"POLYGON ((-94243.895 1515549.915, -94206.484 1...","{11: 582443, 21: 810330, 22: 488479, 23: 32774...","{'Open Water': 582443, 'Developed, Open Space'..."
3,None,41740,310M400US41740,41740,"San Diego-Carlsbad, CA",M1,10900649391,820695499,"POLYGON ((-1978658.296 1372311.247, -1978186.0...","{11: 105190, 21: 715430, 22: 521232, 23: 78493...","{'Open Water': 105190, 'Developed, Open Space'..."
4,340,38220,310M400US38220,38220,"Pine Bluff, AR",M1,5255934204,143446207,"POLYGON ((323220.425 1221669.427, 323225.761 1...","{11: 128879, 21: 136371, 22: 147251, 23: 55086...","{'Open Water': 128879, 'Developed, Open Space'..."
...,...,...,...,...,...,...,...,...,...,...,...
385,None,42020,310M400US42020,42020,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",M1,8548611925,815519172,"POLYGON ((-2243719.665 1716801.604, -2234872.9...","{11: 36005, 21: 393470, 22: 108356, 23: 79455,...","{'Open Water': 36005, 'Developed, Open Space':..."
386,None,49740,310M400US49740,49740,"Yuma, AZ",M1,14281127830,13326079,"POLYGON ((-1746851.979 1221914.112, -1746649.6...","{11: 14767, 21: 127785, 22: 83322, 23: 80360, ...","{'Open Water': 14767, 'Developed, Open Space':..."
387,222,20020,310M400US20020,20020,"Dothan, AL",M1,4444678880,33248036,"POLYGON ((929857.794 949864.962, 934746.562 95...","{11: 47386, 21: 209985, 22: 129704, 23: 46257,...","{'Open Water': 47386, 'Developed, Open Space':..."
388,408,14860,310M400US14860,14860,"Bridgeport-Stamford-Norwalk, CT",M1,1618651428,549293518,"MULTIPOLYGON (((1879105.711 2241432.869, 18791...","{11: 78969, 21: 341906, 22: 242243, 23: 179037...","{'Open Water': 78969, 'Developed, Open Space':..."


In [8]:
import pandas as pd

# Assuming msa_projected is your DataFrame

# Convert the 'land_cover_stats' column to a DataFrame of dictionaries
land_cover_df = pd.json_normalize(msa_projected['land_cover_stats'])

# Add the 'NAME' column to the land cover DataFrame for better readability
land_cover_df['NAME'] = msa_projected['NAME']

# Now, calculate the area of different land cover types across all rows
# This sums the areas for each land cover type across all MSAs
land_cover_area = land_cover_df.drop(columns='NAME').sum()

# Convert to a DataFrame for easier viewing
land_cover_area_df = land_cover_area.reset_index()
land_cover_area_df.columns = ['Land Cover Type', 'Total Area']

# Display the results
print(land_cover_area_df)


   Land Cover Type    Total Area
0               11  7.052739e+07
1               21  1.197171e+08
2               22  9.966774e+07
3               23  7.197881e+07
4               24  2.774634e+07
5               31  4.541894e+07
6               41  2.935452e+08
7               42  2.955902e+08
8               43  1.068815e+08
9               52  5.448734e+08
10              71  2.853483e+08
11              81  2.117424e+08
12              82  4.140613e+08
13              90  1.528717e+08
14              95  5.784511e+07
15            mean  2.171512e+04
16           count  2.798328e+09
17             sum  1.572152e+11
18              12  5.122600e+05


In [9]:
import pandas as pd

# Initialize a list to store results
results = []

# Iterate over each row of msa_projected
for _, row in msa_projected.iterrows():
    msa = row['NAME']  # Get the MSA name
    land_cover = row['land_cover_descriptions']  # Access the land cover descriptions (a dictionary)
    
    # Add each land cover type and area to the results
    for cover_type, area in land_cover.items():
        results.append({'MSA': msa, 'Land Cover Type': cover_type, 'Area': area})

# Convert the results list to a DataFrame
land_cover_areas = pd.DataFrame(results)

# Optional: Aggregate to sum up areas for each MSA and Land Cover Type
aggregated_areas = land_cover_areas.groupby(['MSA', 'Land Cover Type']).sum().reset_index()

# Display the final DataFrame
aggregated_areas


,MSA,Land Cover Type,Area
0,"Abilene, TX",Barren Land (Rock/Sand/Clay),6728
1,"Abilene, TX",Cultivated Crops,2099812
2,"Abilene, TX",Deciduous Forest,153889
3,"Abilene, TX","Developed, High Intensity",27003
4,"Abilene, TX","Developed, Low Intensity",108973
...,...,...,...
5674,"Yuma, AZ",Mixed Forest,206
5675,"Yuma, AZ",Open Water,14767
5676,"Yuma, AZ",Pasture/Hay,17333
5677,"Yuma, AZ",Shrubland,11085891


In [10]:
# Assuming your DataFrame is named land_cover_areas
pivoted_df = aggregated_areas.pivot_table(
    index='MSA',           # Use MSA as the row index
    columns='Land Cover Type',  # Use Land Cover Type as the columns
    values='Area',         # Use Area as the values
    aggfunc='sum',         # Aggregate values using sum in case of duplicates
).fillna(0)               # Replace NaN with 0 (no area for a type)

# Reset index to make MSA a column
pivoted_df = pivoted_df.reset_index()

pivoted_df

pivoted_df.to_csv('../../data/tidy/msa-land-cover-area.csv', index=False)

In [11]:
# land cover comparison

# Compare total area of different land cover types across all MSAs
land_cover_area_comparison = land_cover_df.sum(axis=0)

print(land_cover_area_comparison)


11                                              70527389.0
21                                             119717071.0
22                                              99667736.0
23                                              71978812.0
24                                              27746345.0
31                                              45418940.0
41                                             293545214.0
42                                             295590244.0
43                                             106881529.0
52                                             544873430.0
71                                             285348328.0
81                                             211742363.0
82                                             414061319.0
90                                             152871726.0
95                                              57845113.0
mean                                           21715.11806
count                                           27983278

In [12]:
import numpy as np

# Calculate the proportions of each land cover type
land_cover_proportions = land_cover_df.div(land_cover_df.sum(axis=1), axis=0)

# Calculate Shannon entropy for each MSA
entropy = land_cover_proportions.apply(lambda row: -np.sum(row * np.log(row + 1e-9)), axis=1)

# Add the entropy value to the DataFrame
msa_projected['land_cover_entropy'] = entropy

print(msa_projected[['NAME', 'land_cover_entropy']])


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
import pandas as pd
import numpy as np

# Convert all columns to numeric, forcing errors to NaN (useful if there are unexpected non-numeric values)
land_cover_df = land_cover_df.apply(pd.to_numeric, errors='coerce')

# Now, calculate the proportions of each land cover type
land_cover_proportions = land_cover_df.div(land_cover_df.sum(axis=1), axis=0)

# Calculate Shannon entropy for each MSA
entropy = land_cover_proportions.apply(lambda row: -np.sum(row * np.log(row + 1e-9)), axis=1)

# Display the entropy values for each MSA
print(entropy)
